In [ ]:
#@title Download stuff
# Run this cell to load everything :)
!echo Downloading stuff...
!wget 'https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/Deep%20Dives/Advanced%20Topics%20in%20AI/Sessions%206%20-%2010%20(Projects)/Project%20-%20Self%20Driving%20Cars/rl.zip' -O temp > /dev/null 2>&1
!unzip temp -d rl > /dev/null 2>&1

!wget 'https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/Deep%20Dives/Advanced%20Topics%20in%20AI/Sessions%206%20-%2010%20(Projects)/Project%20-%20Self%20Driving%20Cars/training_results.zip' -O training_results.zip > /dev/null 2>&1
!unzip training_results.zip -d training_results > /dev/null 2>&1

!wget 'https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/Deep%20Dives/Advanced%20Topics%20in%20AI/Sessions%206%20-%2010%20(Projects)/Project%20-%20Self%20Driving%20Cars/videos.zip' -O videos.zip > /dev/null 2>&1
!unzip videos.zip -d videos > /dev/null 2>&1

!wget 'https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/Deep%20Dives/Advanced%20Topics%20in%20AI/Sessions%206%20-%2010%20(Projects)/Project%20-%20Self%20Driving%20Cars/checkpoints.zip' -O checkpoints.zip > /dev/null 2>&1
!unzip checkpoints.zip -d checkpoints > /dev/null 2>&1


!echo Installing packages... 

!pip install Box2D > /dev/null 2>&1
!pip install gym[box2d] pyvirtualdisplay > /dev/null 2>&1
!apt-get update > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!pip install plotly --upgrade > /dev/null 2>&1

!echo Done!
!ls

Installing packages...
Done!
checkpoints	 sample_data	   training_results.zip
checkpoints.zip  temp		   videos
rl		 training_results  videos.zip


# Driving and Reinforcement Learning

Last time we reduce the driving problem to mimicing a human driver (behavior cloning). This then became the supervised learning problem of predicting actions from observations. While we saw that this performed at a decent level and was able to drive around the track, we also noted some important drawbacks of this approach.

Today, we'll look at a different approach to solving the same problem - reinforcement learning. In partcular, we'll see how we can teach a driver how to drive without explicitly giving it any human guidance! The AI will learn through repeated trial and error!

Let's get started!

In [ ]:
#@title Load libraries
# %load_ext autoreload
# %autoreload 2
# %load_ext tensorboard

INDEX2STRING = [
    "stay",
    "accelerate",
    "right",
    "left",
]

import gym 
import plotly.express as px
import numpy as np
import warnings
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from collections import Counter, defaultdict
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import datetime
from sklearn.metrics import confusion_matrix
import plotly.figure_factory as ff
from tqdm import tqdm

import os
import pickle
import random
import sys

import gym
import numpy as np
import plotly.express as px
from gym.wrappers import Monitor
from tensorflow.keras.layers import (
    Activation,
    BatchNormalization,
    Conv2D,
    Dense,
    Dropout,
    Flatten,
    Lambda,
    MaxPool2D,
    Permute,
    Reshape,
)
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import json 
from rl.agents.dqn import DQNAgent
from rl.callbacks import Callback, FileLogger, ModelIntervalCheckpoint
from rl.memory import SequentialMemory
from rl.policy import BoltzmannQPolicy, EpsGreedyQPolicy, LinearAnnealedPolicy, Policy
from rl.processors import Processor

import glob
from IPython import display as ipythondisplay
from tqdm.notebook import tqdm
from gym.wrappers import Monitor
from IPython.display import HTML
import base64
import io
import pickle
warnings.filterwarnings("ignore")
from pyvirtualdisplay import Display

display = Display(visible=0, size=(1400, 900))
display.start()


INDEX2STRING = [
    "stay",
    "accelerate",
    "right",
    "left",
]
INDEX2ACTIONS = [
    (0, 0, 0),
    (0, 1, 0),
    (1, 0, 0),
    (-1, 0, 0),
]

def run_blackjack(policy, show=False):
    env = gym.make("Blackjack-v0")
    obs = env.reset()
    history = []
    while True:
        move = policy(obs)
        new_obs, reward, done, info = env.step(move)
        history.append((obs, move, new_obs, done, reward))
        obs = new_obs
        if done:
            break
    if show:
        result = history[-1][-1]
        if result > 0:
            print("Congrats! You win")
        if result == 0:
            print("Tie!")
        if result < 0:
            print("You lose!")
        print("Your cards: ", env.player)
        print("Dealers cards", env.dealer)

    env.close()
    return history

def human_blackjack_player(obs):
    print(f"Your total: {obs[0]}")
    print(f"Dealer card: {obs[1]}")
    print(f"Usable ace?: {obs[2]}")
    return int(input("0 for stay 1 for hit: "))


def load_pkl(fname):
    with open(fname, "rb") as f:
        return pickle.load(f)
"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")

def play_mp4(mp4):
  video = io.open(mp4, 'r+b').read()
  encoded = base64.b64encode(video)
  ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env


def run_racing(policy, max_steps=1000):
    env = gym.make("CarRacing-v0")
    env = wrap_env(env)
    obs = env.reset() # Intialize the observation
    for step in tqdm(range(min(max_steps, 1000))):
        env.render()

        move = policy(obs, env, step)
        obs, reward, done, info = env.step(move)
        if done:
            break

    env.close()
    show_video()

def load_json(fname):
    with open(fname, "r") as f:
        return json.load(f)


## Q-Learning

We'll use a famous RL algorithm called Q-Learning. Before we apply Q-Learning to the complex problem of driving, let's try to get a handle on the algorithm itself! Here we'll use the words _state_ and *observation* interchangeably.

Recall we have the following setup: 

![setup](https://www.novatec-gmbh.de/wp-content/uploads/1_mPGk9WTNNvp3i4-9JFgD3w.png)

### What is Q?
Q is a function that determines the **quality** of a certain (state, action) pair. Or an estimate of the best possible total reward I get for taking an action at a certain state. So Q(s, a) gives me the quality of taking action $a$ at state $s$. 

**Question:** If I have such a function, what policy should I use at a certain state?




In [ ]:
#@title Double click here for idea

''' 
If I'm at state $s$, I can use $Q$ to check which action $a$ has the highest quality, and then choose that action to perform!
Mathematically, I want to pick the action $a$ that maximizes $Q(s, a)$.
'''


" \nIf I'm at state $s$, I can use $Q$ to check which action $a$ has the highest quality, and then choose that action to perform!\nMathematically, I want to pick the action $a$ that maximizes $Q(s, a)$.\n"

**Exercise:** Write this policy! Let's imagine that there are only two possible actions 0 and 1. 
* Access the $Q(s, a)$ using $q[(s, a)]$

In [ ]:
def q_policy(s, q):
    ## BEGIN CODE HERE
    return 0 if q[(s, 0)] >= q[(s, 1)] else 1
    '''
    max_val = None
    max_action = None
    for state, action in q:
      if state == s:
        if max_val is None or max_val < q[(s,action)]:
          max_val = q[(s,action)]
          max_action = action
    return max_action
    '''
    ## END CODE HERE

    # return 0 if q[(obs, 0)] >= q[(obs, 1)] else 1



Awesome! The tricky part is then learning the $Q$ function itself (hence the name $Q$-learning) The idea is just to run a bunch of random simulations and depending on the reward we get at each step update our quality values for certain (state, action) pairs. In particular, we update the q-values using the following equation:

$$Q_{new}(s_t, a_t) = (1-\alpha)Q_{old}(s_t,a_t) + \alpha[r_t + \gamma \max_a(Q_{old}(s_{t+1}, a))]$$

That's a big equation so let's break it down:
* $Q_{new}(s_t, a_t)$ is our updated estimate for the quality of a certain (state, action) pair
* $Q_{old}(s_t,a_t)$ is our old estimate for the 
* $r_t + \gamma \max_a(Q_{old}(s_{t+1}, a))$. This gives a new estimate of $Q(s, a)$ based on a simulation and our prior estimate. It is calculated as  the reward that we get for taking $a_t$ at state $s_t$ plus the best possible total reward I can get at the next state using our old estimates. $\gamma$ is known as a discount factor which dictates how much we care about rewards in the distant future as compared to rewards now.
* $\alpha$ is our learning rate which dictates how much we want to update our beliefs on the for each simulation.

Rephrased: The above equation updates our current estimate of $Q(s, a)$ by shifting it towards our new estimate from simulations by a factor of $\alpha$.

Don't worry if that all seems confusing we'll implement Q-learning ourselves from scratch!

## Blackjack

To get the hang of Q-learning, let's use it to play blackjack! Here are some quick rules:
* You start of with two cards
* You can 'hit' which is to draw another card. Or 'stay' which is to say you don't want more cards
* The goal is to have your cards add up to larger than the dealers cards but less than 21.
* You can see one of the dealers cards
* An ace can count as either 1 or 11 depending on which gives you the higher score

The **observations** have the following form (my total, dealer card, usable ace)

The **actions** are either 'hit' or 'stay'

The **reward** is 1 for a win, 0 for a tie and -1 for a draw.

Let's play a couple of rounds!

In [ ]:
history = run_blackjack(human_blackjack_player, show=True)

Your total: 16
Dealer card: 5
Usable ace?: False
0 for stay 1 for hit: 1
Your total: 21
Dealer card: 5
Usable ace?: False
0 for stay 1 for hit: 0
Congrats! You win
Your cards:  [6, 10, 5]
Dealers cards [5, 1, 8, 10]


Cool! Now that you have some intuition for the game, let's look at what data we record for each simulation.

In [ ]:
print(history[0])

((16, 5, False), 1, (21, 5, False), False, 0.0)


This is one chunk of our experience represented as a tuple. Here it is printed out nicely.

In [ ]:
def print_hist(h):
  print(f"The state was: {h[0]}")
  print(f"The action was: {h[1]}")
  print(f"The state is now: {h[2]}")
  print(f"The episode is over: : {h[3]}")
  print(f"Reward for this action: {h[4]}")

print_hist(history[0])

The state was: (16, 5, False)
The action was: 1
The state is now: (21, 5, False)
The episode is over: : False
Reward for this action: 0.0


The elements of the tuple are the following 
1. $s_t$
2. $a_t$ (hit or stay)
3. $s_{t+1}$ the state we end up in after taking the action
4. True if the episode is over, False otherwize.
5. $r_t$ reward at this time step

Given this information, we can then use our equation to update our estimates for $Q$! As it is tiring to play many games of blackjack, we instead let our q-learning agent play by itself against the dealer, with the hopes that it learns from its experience!

## Exploration vs Exploitation
When the q-learning agent plays, what policy should it use in order to get the most experience? 

**Question:** Think back to when you first stated to play a certain game, what did you do in order to get better?

Maybe it was a mix of trying new strategies and using the ones that worked and fine tuning them! That's exactly what we'll do here! We'll use a policy called the $\epsilon$-greedy policy, which tries new (random strategies) with probability $\epsilon$ and uses strategies that it knows will work the rest of the time!

**Exercise:** Write the epsilon greedy policy: 
* `eps` is the probability we select a random action
* `random.random()` gives us a random number between 0 and 1
* How can we use the above to do one thing with probabiliy $\epsilon$?
* `random.choice([0,1])` gives us a random action
* remember we wrote a function `q_policy(s, q)` above which gives us the best action to take given our estimate for $Q$.

In [ ]:
def eps_policy(s, q, eps):
    ## BEGIN CODE HERE
    if random.random() <= eps:
      return random.choice([0,1])
    else:
      return q_policy(s, q)
    ## END CODE HERE


In [ ]:
def q_learn_blackjack(eps, gamma, alpha, epochs):
    q_table = defaultdict(int)
    training_results = []
    for _ in tqdm(range(epochs)):
        # history contains the (s, a, s_next, done, r) tuples that we saw ealier for a particular run of the game
        history = run_blackjack(lambda obs: eps_policy(obs, q_table, eps))
        for s, a, s_next, done, r in reversed(history):
            old = q_table[(s, a)]
            if done:  # Don't care about the best next b/c we're done!
                q_table[(s, a)] = (1 - alpha) * old + alpha * (r)
            else:
                best_next = max([q_table[(s_next, 0)], q_table[(s_next, 1)]])
                q_table[(s, a)] = (1 - alpha) * old + alpha * (r + gamma * best_next)
        training_results.append(history[-1][-1])
    return q_table, training_results

Run the cell below to train the q-learning agent for blackjack! It'll take a while for it to train. While it trains, take a look at the code above and try to understand how this code works and implements the q-learning algorithm we described. Ask your instructor questions!

In [ ]:
EPS = 0.7
GAMMA = 1
ALPHA = 0.2
EPOCHS = 50000
q_table, results = q_learn_blackjack(EPS, GAMMA, ALPHA, EPOCHS)

  0%|          | 0/50000 [00:00<?, ?it/s]

Great! Now that that's done training, let's check out what it learned. Let's print out the q-table! This shows us the quality of each more at each state that the q-learning algorithm encountered. Remember that it maps `(my_total, dealer_card, usable_ace) , hit(1)/stay(0))` to the quality of taking that action in that observation. 



In [ ]:
q_table

defaultdict(int,
            {((4, 1, False), 0): -0.595474059264,
             ((4, 1, False), 1): -0.3138238044224696,
             ((4, 2, False), 0): -0.03595786240000001,
             ((4, 2, False), 1): -0.045794114471471054,
             ((4, 3, False), 0): -0.14157457408000004,
             ((4, 3, False), 1): -0.1662053080538058,
             ((4, 4, False), 0): 0.31532482969600006,
             ((4, 4, False), 1): 0.08073951400366114,
             ((4, 5, False), 0): -0.1656448,
             ((4, 5, False), 1): 0.0627916437892701,
             ((4, 6, False), 0): -0.26296821760000005,
             ((4, 6, False), 1): 0.11283304643446877,
             ((4, 7, False), 0): -0.7378560000000001,
             ((4, 7, False), 1): 0.0344555473423432,
             ((4, 8, False), 0): -0.5740160000000001,
             ((4, 8, False), 1): -0.19925012964307662,
             ((4, 9, False), 0): -0.6264448,
             ((4, 9, False), 1): -0.29388211318885965,
             ((4, 10, False)

**Discuss**: Do the values make sense and why/why not?

Great! Now that we have our q table, we can use the `q_policy` function that we wrote earlier to play blackjack! Run the cell below a couple of times to see it play!

In [ ]:
run_blackjack(lambda s : q_policy(s, q_table), show=1)

You lose!
Your cards:  [3, 9, 10]
Dealers cards [6, 5]


[((12, 6, False), 1, (22, 6, False), True, -1.0)]

# Question: What's the difference between Blackjack and Driving?

Which one seems harder?



Let's do a estimation exercise. Let's estimate/count/upperbound the size of observation space for blackjack and for driving. As a reminder the observations are the following:
* blackjack: `(my_count, dealer_card, usable_ace)`.
* driving: `96x96x3` images, where each pixel takes a value between 0 and 255. 

**Exercise:**  Give your best guess for either! Try to get as close as possible and argue why your answer is good! Note that instructors don't have the answer for this!

Then we have 2 actions for blackjack and 4 actions for driving. Given this and your estimates before, compare the size of the q-tables for blackjack vs. driving.

## The problem with Q-learning

Our q-learning policy works by looking up the q-table at a certain state, and picking the best action. But this assumes that we have encountered this state before and have tried multiple actions during training in order to compare the quality of those actions. As we have seen, the size of the q-table for driving would be orders of magnitude larger than for blackjack! It is **computationally infeasible** to do so many simulations to see find each of the states try each action multiple times to get a good estimate of the q-values.

To address this we do something really clever called **function approximation**. Instead of building a q-table, we build a q-approximator, which in this case will be a neural network. The function approximator looks like this. 

![func approx](https://media.geeksforgeeks.org/wp-content/uploads/20190613104538/Deep-Q-Learning.png)

The neural network takes as input an observation and then gives a prediction for the q-values for each action!

**Question:** How does this get around the problem that there are so many states?

Approximating the Q-function with a neural networks is a technique called Deep-Q-Learning, and it was first used to learn how to play Atari games at super human level by researchers at DeepMind! Check [this](https://www.youtube.com/watch?v=V1eYniJ0Rnk&vl=en) out after the session!

# Deep Q-Learning

In [ ]:
#@title Run to load some helpers
ENV_NAME = "CarRacing-v0"
INPUT_SHAPE = (96, 96, 3)
WINDOW_LENGTH = 3
VIDEO_INTERVAL = 25 # in episodes
TRAIN_STEPS = 175000
ANNEAL_STEPS = 100000
NB_ACTIONS = 4


class RacingProcessor(Processor):
    def __init__(self, env):
        self.env = env
        self.count = 0

    def process_action(self, action):
        return INDEX2ACTIONS[action]


def start_policy(state):
    return 1

def get_driving_env_dqn(name, grass_penalty, still_penalty):
    env = Monitor(RewardWrapper(gym.make(ENV_NAME), grass_penalty, still_penalty), f"/home/harry/Documents/videos/{name}", lambda x: x % VIDEO_INTERVAL==0, force=True)
    return env

NB_ACTIONS = 4

# Next, we build a very simple model.
# model = Sequential(
#     [
#         Permute((2, 3, 1, 4), input_shape=(WINDOW_LENGTH, 96, 96, 3)),
#         Reshape((96, 96, -1)),
#         Lambda(lambda x: x / 255.0),
#         MaxPool2D(2),
#         Conv2D(32, 3, activation="relu"),
#         MaxPool2D(2),
#         # Dropout(0.2),
#         Conv2D(64, 3, activation="relu"),
#         MaxPool2D(2),
#         # Dropout(0.2),
#         Conv2D(64, 3, activation="relu"),
#         Flatten(),
#         Dense(32, activation="relu"),
#         Dense(NB_ACTIONS),
#     ]
# )
# print(model.summary())

# Finally, we configure and compile our agent. You can use every built-in tensorflow.keras optimizer and
# even the metrics!
memory = SequentialMemory(limit=50000, window_length=WINDOW_LENGTH)
# policy = BoltzmannQPolicy()
annealed_policy = LinearAnnealedPolicy(
    EpsGreedyQPolicy(),
    attr="eps",
    value_max=1,
    value_min=0.1,
    value_test=0.05,
    nb_steps=ANNEAL_STEPS,
)
eps_greedy_policy = EpsGreedyQPolicy(0.1)

small_eps = EpsGreedyQPolicy(0)


def get_dqn(env, policy, model, test_policy='regular'):
    if test_policy == 'regular':
      tp = small_eps
    else:
      tp = SpeedLimitPolicy(env, 3000)
    dqn = DQNAgent(
        model=model,
        nb_actions=NB_ACTIONS,
        policy=policy,
        memory=memory,
        processor=RacingProcessor(env),
        nb_steps_warmup=100,
        gamma=0.95,
        target_model_update=10000,
        train_interval=4,
        # delta_clip=1.0,
        test_policy=tp,
        enable_dueling_network=True
    )

    dqn.compile(Adam(learning_rate=1e-4), metrics=["mae"])
    return dqn


# Okay, now it's time to learn something! We visualize the training here for show, but this
# slows down training quite a lot. You can always safely abort the training prematurely using
# Ctrl + C.


def load(dqn, fname):
    dqn.load_weights(fname)
    return dqn


def train(env, dqn, name, action_rep):
    # log_path = f"driving_models/{name}.log"
    # checkpoint_path= f"/home/harry/Documents/driving_models/{name}.cpt"
    callbacks = [
        # ModelIntervalCheckpoint(checkpoint_path, interval=10000),
        # FileLogger(log_path, interval=1000),
    ]
    history = dqn.fit(
        env,
        nb_steps=TRAIN_STEPS,
        visualize=True,
        verbose=2,
        action_repetition=action_rep,
        nb_max_episode_steps=None,
        callbacks=callbacks,
    )
    # dqn.save_weights(checkpoint_path, overwrite=True)

class SpeedLimitPolicy(Policy):
    """Implement the greedy policy

    Greedy policy returns the current best action according to q_values
    """

    def __init__(self, env, max_speed, eps=0):
        self.max_speed = max_speed
        self.env = env
        self.eps = eps

    def select_action(self, q_values):
        """Return the selected action

        # Arguments
            q_values (np.ndarray): List of the estimations of Q for each action

        # Returns
            Selection action
        """
        if np.random.uniform() < self.eps:
            return np.random.randint(0, NB_ACTIONS)
        else:
            speed = get_features_from_env(env)[0]
            order = np.argsort(q_values)
            best = order[-1]
            if speed > self.max_speed and best == 1:
                return order[-2]
            else:
                return best

def get_videos(name):
  videos = sorted([f'videos/{name}/{x}' for x in os.listdir(f"videos/{name}") if 'mp4' in x])
  print(f"There are {len(videos)} recorded video episodes")
  return(videos)

# Finally, evaluate our algorithm for 5 episodes.
def test(env, dqn, name):
    checkpoint_path= f"checkpoints/{name}.cpt"
    dqn = load(dqn, checkpoint_path)
    dqn.test(
        env,
        nb_episodes=1,
        visualize=False,
        action_repetition=4,
        # nb_max_start_steps=40,
        # start_step_policy=start_policy,
        nb_max_episode_steps=None,
    )

def get_features_from_env(env):
    wheel = env.car.wheels[0]
    speed = sum(map(lambda x: x ** 2, list(wheel.linearVelocity)))
    return [speed, wheel.angularVelocity] + list(wheel.linearVelocity)

We need three things for a Deep-Q-Learning agent 
1. An environment to give use observations/rewards/actions
2. A policy used to explore our actions
3. A neural network used to approximate the Q-Function!

The policy we are going to use here is a slighly modified version of the epsilon greedy policy. Instead of setting epsilon as a constant, we have epsilon change gradually from 1 to 0.1 as the training progresses

Question: 
* Why would reducing epsilon over time be desirable?

In [ ]:
ENV_NAME = "CarRacing-v0"
INPUT_SHAPE = (96, 96, 3)
WINDOW_LENGTH = 3
VIDEO_INTERVAL = 25  # in episodes
TRAIN_STEPS = 175000
ANNEAL_STEPS = 100000
env = gym.make('CarRacing-v0')
env = wrap_env(env)

annealed_policy = LinearAnnealedPolicy(
    EpsGreedyQPolicy(),
    attr="eps",
    value_max=1,
    value_min=0.1,
    value_test=0.05,
    nb_steps=ANNEAL_STEPS,
)

model = Sequential(
    [
        Permute((2, 3, 1, 4), input_shape=(WINDOW_LENGTH, 96, 96, 3)), #(h, w, window, rgb)
        Reshape((96, 96, -1)), # Flatten the window dimension
        Lambda(lambda x: x / 255.0), # Normalize
        MaxPool2D(2), # Compress image (yes I know this is GPU inefficient)
        Conv2D(32, 3, activation="relu"),
        MaxPool2D(2),
        # Dropout(0.2),
        Conv2D(64, 3, activation="relu"),
        MaxPool2D(2),
        # Dropout(0.2),
        Conv2D(64, 3, activation="relu"),
        Flatten(),
        Dense(32, activation="relu"),
        Dense(NB_ACTIONS),
    ]
)
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 permute (Permute)           (None, 96, 96, 3, 3)      0         
                                                                 
 reshape (Reshape)           (None, 96, 96, 9)         0         
                                                                 
 lambda (Lambda)             (None, 96, 96, 9)         0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 48, 48, 9)        0         
 )                                                               
                                                                 
 conv2d (Conv2D)             (None, 46, 46, 32)        2624      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 23, 23, 32)       0         
 2D)                                                    

Deep Q-Learning systems take a lot longer to train than our Behavior Cloning model from last time because we need to both generate data and train at the same time! Also, our agent doesn't start off with ANY knowledge of how to drive it literally just random chooses actions! Whereas, last time we gave it human examples of good driving to learn off of, this one is really learning from scratch!

So we won't train the model here. Instead we have included some videos of how the model performes during training. Is our model learning well?

**For reference when I play, I average around 750.**

In [ ]:
dqn = get_dqn(env, annealed_policy, model)

In [ ]:
baseline_videos = get_videos("baseline_repeat")

There are 12 recorded video episodes


In [ ]:
play_mp4(baseline_videos[0])

Note that by the end of the training, we still are using some randomness in our policy. Let's load up the trianed model and see how it does when choosing the action it thinks is best all the time!

In [ ]:
test(env, dqn, 'baseline_repeat')
env.close()
show_video()

Testing for 1 episodes ...
Track generation: 1198..1501 -> 303-tiles track
Episode 1: reward: -113.587, steps: 117


 Hmm it's not great is it? Let's visualize the training process. Double click the legend to show individual plots

In [ ]:
x  = load_json('training_results/baseline_repeat.log')
px.line(pd.DataFrame(x))

## Safety first!
One thing you may have noticed is that there are some instances where the car veered off the track! This is very bad if we want our self driving car to be on the road! To make sure our car is safe, we can tweak the environment to penalize going off the road. 

Complete the code below!

`left_col` is the color of a pixel just left of the car 
`right_col` is the color of a pixel just right of the car 

return -1000 if either the `left_col` is not a road color or `right_col` is not a road color. 
o/w return the original reward `rew`. 

In [ ]:
ROAD_COLORS = [(102 + i, 102 + i, 102 + i) for i in range(6)]

class PenalizeLeavingRoad(gym.RewardWrapper):
    def __init__(self, env):
        super().__init__(env)


    def reward(self, rew):
        left_col = tuple(self.env.state[65, 50, :])
        right_col = tuple(self.env.state[65, 45, :])
        ### BEGIN CODE HERE
        if left_col not in ROAD_COLORS or right_col not in ROAD_COLORS:
          return -1000
        else:
          return rew
        ### END CODE HERE


\Ask your instructor to copy this code and test it out to see it if it works as expected. TODO
[Instructors checkout the setup doc]

Visualize some training examples! How did the car do?

In [ ]:
safe_videos = get_videos("safe2")

There are 33 recorded video episodes


In [ ]:
play_mp4(safe_videos[32]) # TODO: try 

Here are the plots:

In [ ]:
x  = load_json('training_results/safe2.log')
px.line(pd.DataFrame(x))

Let's also run a trained version of this model!!

In [ ]:
env = wrap_env(gym.make("CarRacing-v0"))
dqn = get_dqn(env, annealed_policy, model)
test(env, dqn, 'safe2')
env.close()
show_video()

Testing for 1 episodes ...
Track generation: 1206..1512 -> 306-tiles track
Episode 1: reward: 559.016, steps: 250


That's significantly better! But it's still going a little slow. Let's modify the reward wrapper from last time to also penalize staying still, so that it actually moves more!


This time, first check if the car is off the road, if it is, return -1000, otherwize use the `is_still(rew)` function to deterine if the car is still, if it is, return -0.1, otherwize return the original reward

In [ ]:
class PenalizeNotMoving(gym.RewardWrapper):
    def __init__(self, env, grass_penalty, still_penalty):
        super().__init__(env)

    def reward(self, rew):
        left_col = tuple(self.env.state[65, 50, :])
        right_col = tuple(self.env.state[65, 45, :])
        if left_col not in ROAD or right_col not in ROAD:
            return -1000
        if is_still(rew):
            return -0.1
        return rew

To test this code, ask the instructor to copy and paste this and run the driving simulator a couple of times!

Let's see how training this one look like!

In [ ]:
mix_videos = get_videos("mix")

There are 35 recorded video episodes


In [ ]:
play_mp4(mix_videos[31])

NameError: ignored

Here are the plots:

In [ ]:
x  = load_json('training_results/mix.log')
px.line(pd.DataFrame(x))

Let's also run a trained version of this model!!

In [ ]:
env = wrap_env(gym.make("CarRacing-v0"))
dqn = get_dqn(env, annealed_policy, model)
test(env, dqn, 'mix')
env.close()
show_video()

How is the model doing? Run it a couple of times to see how it performs on random tracks!

## Conclusion

Question: What did we accomplish in this notebook and what technqiues did we use?
* Q-Learning
* Deep-Q-Learning
* Tweaking reward functions







## Acknoledgements
* [Open AI](https://gym.openai.com/docs/) for providing the enviroment
* [Tensorflow](https://www.tensorflow.org/) for the Neural Networks
* [Keras RL2](https://github.com/wau/keras-rl2) for the implementation of DQN

Project created by Harry Sha